In [3]:
import sys
import torch
from collections import Counter

sys.path.append("../")

In [4]:
from data import load_gyafc
from torch.utils.data import Dataset, DataLoader
from transformers import (AutoModelForSequenceClassification,  
                          AutoTokenizer)
from sklearn.metrics import classification_report

In [5]:
datasets = load_gyafc("microsoft/deberta-base")

In [6]:
_, _, test_dataset = datasets

In [7]:
test_dataloader = DataLoader(
            test_dataset,
            batch_size=64,
            # num_workers=4,
            drop_last=True
        )

In [6]:
md = "microsoft_deberta-base_ep5_wus1000_lr1e-06"

In [7]:
trained_model = f"../trained_models/{md}/nli_model/"

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(trained_model)
tokenizer = AutoTokenizer.from_pretrained(trained_model)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [10]:
list_predited_label = []
list_label = []
with torch.no_grad():
    for d in test_dataloader:
        input_ids = d["input_ids"].to(device) # .reshape(64, 24)
        attention_mask = d["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask)
        logits = outputs[0]

        _, prediction = torch.max(logits, dim=1)
        targets = d["labels"].detach().numpy().tolist()
        prediction = prediction.cpu().detach().numpy().tolist()

        list_label.extend(targets)
        list_predited_label.extend(prediction)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [11]:
list_predited_label_1 = []
list_label = []
with torch.no_grad():
    for d in test_dataloader:
        input_ids = d["input_ids"].to(device) # .reshape(64, 24)
        attention_mask = d["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask)
        logits = outputs[0]

        _, prediction = torch.max(logits, dim=1)
        targets = d["labels"].detach().numpy().tolist()
        prediction = prediction.cpu().detach().numpy().tolist()

        list_label.extend(targets)
        list_predited_label_1.extend(prediction)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [12]:
check_stupid = [a==b for a,b in zip(list_predited_label, list_predited_label_1)]
Counter(check_stupid)

Counter({True: 41600})

In [13]:
import json
with open(f"../trained_models/{md}/test_predicts.json") as f:
    list_predited_label_saved = json.load(f)

In [14]:
import json
with open(f"GYAFC_test_labels.json") as f:
    labels_saved = json.load(f)

In [15]:
check_stupid_labels = [a==b for a,b in zip(list_label, labels_saved)]
Counter(check_stupid_labels)

Counter({True: 41600})

In [16]:
# list_predited_label

In [17]:
# list_predited_label_saved

In [18]:
len(list_predited_label), len(list_predited_label_saved)

(41600, 41600)

In [19]:
check = [a==b for a,b in zip(list_predited_label, list_predited_label_saved)]
Counter(check)

Counter({True: 32847, False: 8753})

In [20]:
print(classification_report(list_label, list_predited_label, digits=3))

              precision    recall  f1-score   support

           0      0.818     0.962     0.884     22151
           1      0.946     0.756     0.840     19449

    accuracy                          0.866     41600
   macro avg      0.882     0.859     0.862     41600
weighted avg      0.878     0.866     0.864     41600



In [21]:
print(classification_report(list_label, list_predited_label_saved, digits=3))

              precision    recall  f1-score   support

           0      0.818     0.962     0.884     22151
           1      0.946     0.756     0.840     19449

    accuracy                          0.866     41600
   macro avg      0.882     0.859     0.862     41600
weighted avg      0.878     0.866     0.864     41600



In [22]:
Counter([a==b for a,b in zip(list_predited_label, list_predited_label_saved)])

Counter({True: 32847, False: 8753})

In [23]:
print(classification_report(labels_saved, list_predited_label, digits=3))

              precision    recall  f1-score   support

           0      0.818     0.962     0.884     22151
           1      0.946     0.756     0.840     19449

    accuracy                          0.866     41600
   macro avg      0.882     0.859     0.862     41600
weighted avg      0.878     0.866     0.864     41600



In [24]:
print(classification_report(labels_saved, list_predited_label_saved, digits=3))

              precision    recall  f1-score   support

           0      0.818     0.962     0.884     22151
           1      0.946     0.756     0.840     19449

    accuracy                          0.866     41600
   macro avg      0.882     0.859     0.862     41600
weighted avg      0.878     0.866     0.864     41600



In [ ]:
# for a,b in zip(list_predited_label, list_predited_label_saved):
#     if a!=b:
#         print(a,b)